In [ ]:
# import asyncio
# import nest_asyncio
# import logging
# import pandas as pd
# from collections import deque
# from datetime import datetime
# import pyotp
# import polars as pl
# import numba_indicators 
# from  datetime import timedelta
# from NorenRestApiPy.NorenApi import NorenApi

# # Basic logger setup
# logging.basicConfig(
#     level=logging.INFO,
#     format='%(asctime)s - %(levelname)s - %(message)s',
#     handlers=[logging.StreamHandler()]
# )
# logger = logging.getLogger()

# # Global variables
# api = None
# feed_opened = False
# ring_buffers = {}  # Dictionary to hold ring buffers for each token
# resampled_buffers = {}
# RING_BUFFER_SIZE = 1000  # Example ring buffer size
# RING_BUFFER_RESAMPLE_SIZE = 1000
# resampling_enabled = {}  # Dictionary to track resampling status for each token and timeframe
# VALID_TIMEFRAMES = ['1min', '5min']  # Add or remove timeframes as needed
# last_tick_time = {}  # New: Track the last tick time for each token
# active_subscriptions = set()  # Set to track active subscriptions

# # Initialize API function
# def initialize_api(credentials_file="usercred.xlsx"):
#     global api
#     api = NorenApi(
#         host="https://api.shoonya.com/NorenWClientTP/",
#         websocket="wss://api.shoonya.com/NorenWSTP/"
#     )

#     credentials = pd.read_excel(credentials_file)
#     user = credentials.iloc[0, 0]
#     password = credentials.iloc[0, 1]
#     vendor_code = credentials.iloc[0, 2]
#     app_key = credentials.iloc[0, 3]
#     imei = credentials.iloc[0, 4]
#     qr_code = credentials.iloc[0, 5]
#     factor2 = pyotp.TOTP(qr_code).now()

#     # Perform login
#     api.login_result = api.login(
#         userid=user,
#         password=password,
#         twoFA=factor2,
#         vendor_code=vendor_code,
#         api_secret=app_key,
#         imei=imei
#     )

# # Function to create ring buffers for each token
# def create_ring_buffers(tokens):
#     global ring_buffers
#     for token in tokens:
#         if token not in ring_buffers:
#             ring_buffers[token] = deque(maxlen=RING_BUFFER_SIZE)
#             last_tick_time[token] = None
#             logger.info(f"Created ring buffer for token: {token}")

# def create_resampled_buffers(tokens, timeframes):
#     global resampled_buffers, resampling_enabled
#     for token in tokens:
#         if token not in resampled_buffers:
#             resampled_buffers[token] = {}
#             resampling_enabled[token] = {}
#             logger.info(f"Created resampled buffers entry for token: {token}")
        
#         for timeframe in timeframes:
#             if timeframe not in resampled_buffers[token]:
#                 resampled_buffers[token][timeframe] = deque(maxlen=RING_BUFFER_RESAMPLE_SIZE)
#                 resampling_enabled[token][timeframe] = False  # Default to disabled
#                 logger.info(f"Created resampled buffer for token {token} and timeframe: {timeframe}")

# # Function to enable/disable resampling
# async def set_resampling(token, timeframe, enable):
#     global resampling_enabled
#     if token in resampling_enabled and timeframe in resampling_enabled[token]:
#         resampling_enabled[token][timeframe] = enable
#         logger.info(f"Resampling {'enabled' if enable else 'disabled'} for token {token} and timeframe {timeframe}")
#     else:
#         logger.warning(f"Token {token} or timeframe {timeframe} not found in resampling_enabled")

# # Function to resample ticks
# async def resample_ticks():
#     global resampled_buffers, ring_buffers

#     while True:
#         await asyncio.sleep(1)  # Check every second

#         for token, buffer in ring_buffers.items():
#             if not buffer:
#                 continue

#             df = pd.DataFrame(list(buffer))
#             df['tt'] = pd.to_datetime(df['tt'])
#             df.set_index('tt', inplace=True)

#             resampled = df['ltp'].resample('1min').ohlc()

#             if token not in resampled_buffers:
#                 resampled_buffers[token] = {}
            
#             resampled_buffers[token]['1min'] = deque(
#                 resampled.reset_index().to_dict('records'),
#                 maxlen=RING_BUFFER_RESAMPLE_SIZE
#             )

# # Event handler for feed updates (tick data)
# def event_handler_feed_update(tick_data):
#     try:
#         if 'lp' in tick_data and 'tk' in tick_data:
#             timest = datetime.fromtimestamp(int(tick_data['ft'])).isoformat()
#             token = tick_data['tk']

#             if token in ring_buffers:
#                 # Append tick data to the respective token's ring buffer
#                 ring_buffers[token].append({'ltp': float(tick_data['lp']), 'tt': timest})
#                 last_tick_time[token] = datetime.fromisoformat(timest)
                
#             else:
#                 logger.warning(f"Token {token} not found in ring buffers. Ignoring tick.")
#     except (KeyError, ValueError) as e:
#         logger.error(f"Error processing tick data: {e}")

# # WebSocket connection and subscription handler
# async def connect_and_subscribe():
#     global feed_opened
#     retry_delay = 1  # Initial retry delay in seconds
#     max_retry_delay = 32  # Maximum retry delay in seconds
#     max_retries = 10  # Maximum number of retries
#     retries = 0  # Counter for retries

#     while retries < max_retries:
#         try:
#             api.start_websocket(
#                 order_update_callback=event_handler_order_update,
#                 subscribe_callback=event_handler_feed_update,
#                 socket_open_callback=open_callback,
#                 socket_close_callback=close_callback
#             )
#             await wait_for_feed_open(timeout=30)  # Wait for feed to open with a timeout
#             logger.info("WebSocket connected successfully.")
            
#             # Initial subscription management (adjust as needed)
#             await manage_subscriptions('add', 'MCX|432294')
#             await manage_subscriptions('add', 'NSE|26009')
#             await manage_subscriptions('add', 'NSE|26000')
            
#             retry_delay = 1  # Reset retry delay after successful connection
#             retries = 0  # Reset retry counter
#             await monitor_connection()
#         except Exception as e:
#             logger.error(f"WebSocket connection error: {e}")
#             retries += 1
#             logger.info(f"Reconnecting in {retry_delay} seconds... (Attempt {retries}/{max_retries})")
#             await asyncio.sleep(retry_delay)
#             retry_delay = min(retry_delay * 2, max_retry_delay)  # Exponential backoff

#     if retries >= max_retries:
#         logger.error("Max retries reached. Exiting.")
#         raise Exception("Max retries reached")

# # Manage subscriptions
# async def manage_subscriptions(command, subscription):
#     global active_subscriptions
#     token = subscription.split('|')[1]  # Extract token from subscription string

#     if command == 'add':
#         if subscription not in active_subscriptions:
#             api.subscribe([subscription])  # Subscribe to the new token
#             active_subscriptions.add(subscription)
#             create_ring_buffers([token])  # Create ring buffer for new token
#             create_resampled_buffers([token], VALID_TIMEFRAMES)  # Create resampled buffers for new token
#             logger.info(f"Subscribed to {subscription}")
#             # Once the subscription is done and ring buffers are created, enable resampling
#             for timeframe in VALID_TIMEFRAMES:
#                 await set_resampling(token, timeframe, True)  # Enable resampling dynamically
            
#             logger.info(f"Resampling enabled for token {token} for all valid timeframes.")
#         else:
#             logger.warning(f"Already subscribed to {subscription}")

#     elif command == 'remove':
#         if subscription in active_subscriptions:
#             api.unsubscribe([subscription])  # Unsubscribe from the token
#             active_subscriptions.remove(subscription)
#             # Note: We're not removing buffers here to preserve data. 
#             # You may want to add cleanup logic if needed.
#             logger.info(f"Unsubscribed from {subscription}")
#         else:
#             logger.warning(f"Not subscribed to {subscription}")

# # Wait for WebSocket feed to open
# async def wait_for_feed_open(timeout):
#     global feed_opened
#     start_time = asyncio.get_event_loop().time()
#     while not feed_opened:
#         if asyncio.get_event_loop().time() - start_time > timeout:
#             raise TimeoutError("Timed out waiting for feed to open")
#         await asyncio.sleep(1)

# # Monitor the WebSocket connection
# async def monitor_connection():
#     global feed_opened
#     while True:
#         if not feed_opened:
#             logger.warning("Feed closed unexpectedly. Reconnecting...")
#             raise Exception("Feed closed")
#         await asyncio.sleep(5)  # Check connection status every 5 seconds

# # WebSocket close callback
# def close_callback():
#     global feed_opened
#     feed_opened = False
#     logger.warning("WebSocket connection closed.")
#     logger.info("Attempting to reconnect...")

# # WebSocket open callback
# def open_callback():
#     global feed_opened
#     if not feed_opened:
#         feed_opened = True
#         logger.info('Feed Opened')
#     else:
#         logger.warning('Feed Opened callback called multiple times.')

# # Event handler for order updates
# def event_handler_order_update(data):
#     logger.info(f"Order update: {data}")

# async def main():
#     initialize_api()  # Initialize the API
#     await asyncio.gather(
#         connect_and_subscribe(),  # This can run continuously
#         resample_ticks()           # This will resample ticks concurrently
#     )

# # Event loop setup
# loop = asyncio.get_event_loop()
# loop.set_debug(True)
# if loop.is_running():
#     nest_asyncio.apply()
# asyncio.create_task(main())
# if not loop.is_running():
#     try:
#         loop.run_forever()
#     except KeyboardInterrupt:
#         logger.info("Received exit signal. Cleaning up...")
#     finally:
#         loop.close()
#         logger.info("Event loop closed. Exiting.")

In [ ]:
active_subscriptions

In [ ]:
await manage_subscriptions('add', 'NSE|26000')

In [ ]:
resampling_enabled

In [ ]:
await set_resampling('432294', '5min', False) 

In [ ]:
import asyncio
import logging
from collections import deque
from datetime import datetime, timedelta
import pandas as pd
import pyotp
from NorenRestApiPy.NorenApi import NorenApi

class TickCollector:
    def __init__(self, credentials_file="usercred.xlsx"):
        self.processing_lock = asyncio.Lock()
        self.api = None
        self.feed_opened = False
        self.ring_buffers = {}
        self.resampled_buffers = {}
        self.resampling_enabled = {}
        self.last_tick_time = {}
        self.active_subscriptions = set()
        self.RING_BUFFER_SIZE = 1000
        self.RING_BUFFER_RESAMPLE_SIZE = 1000
        self.VALID_TIMEFRAMES = ['1min', '5min']
        
        self.logger = self._setup_logger()
        self._initialize_api(credentials_file)

    def _setup_logger(self):
        logger = logging.getLogger(__name__)
        logger.setLevel(logging.INFO)
        handler = logging.StreamHandler()
        formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
        handler.setFormatter(formatter)
        logger.addHandler(handler)
        return logger

    def _initialize_api(self, credentials_file):
        self.api = NorenApi(
            host="https://api.shoonya.com/NorenWClientTP/",
            websocket="wss://api.shoonya.com/NorenWSTP/"
        )
        credentials = pd.read_excel(credentials_file)
        user = credentials.iloc[0, 0]
        password = credentials.iloc[0, 1]
        vendor_code = credentials.iloc[0, 2]
        app_key = credentials.iloc[0, 3]
        imei = credentials.iloc[0, 4]
        qr_code = credentials.iloc[0, 5]
        factor2 = pyotp.TOTP(qr_code).now()

        self.api.login_result = self.api.login(
            userid=user,
            password=password,
            twoFA=factor2,
            vendor_code=vendor_code,
            api_secret=app_key,
            imei=imei
        )

    def create_ring_buffers(self, tokens):
        for token in tokens:
            if token not in self.ring_buffers:
                self.ring_buffers[token] = deque(maxlen=self.RING_BUFFER_SIZE)
                self.last_tick_time[token] = None
                self.logger.info(f"Created ring buffer for token: {token}")

    def create_resampled_buffers(self, tokens, timeframes):
        for token in tokens:
            if token not in self.resampled_buffers:
                self.resampled_buffers[token] = {}
                self.resampling_enabled[token] = {}
                self.logger.info(f"Created resampled buffers entry for token: {token}")
            
            for timeframe in timeframes:
                if timeframe not in self.resampled_buffers[token]:
                    self.resampled_buffers[token][timeframe] = deque(maxlen=self.RING_BUFFER_RESAMPLE_SIZE)
                    self.resampling_enabled[token][timeframe] = False
                    self.logger.info(f"Created resampled buffer for token {token} and timeframe: {timeframe}")

    async def set_resampling(self, token, timeframe, enable):
        if token in self.resampling_enabled and timeframe in self.resampling_enabled[token]:
            self.resampling_enabled[token][timeframe] = enable
            self.logger.info(f"Resampling {'enabled' if enable else 'disabled'} for token {token} and timeframe {timeframe}")
        else:
            self.logger.warning(f"Token {token} or timeframe {timeframe} not found in resampling_enabled")


    def event_handler_feed_update(self, tick_data):
        try:
            if 'lp' in tick_data and 'tk' in tick_data:
                timest = datetime.fromtimestamp(int(tick_data['ft'])).isoformat()
                token = tick_data['tk']
                if token in self.ring_buffers:
                    new_tick = {'tt': timest, 'ltp': float(tick_data['lp'])}
                    self.ring_buffers[token].append(new_tick)
                    self.last_tick_time[token] = datetime.fromisoformat(timest)
                    #self.logger.info(f"Added tick to buffer for token {token}. Ring_Buffer: {self.ring_buffers[token]}")
                else:
                    self.logger.warning(f"Token {token} not found in ring buffers. Ignoring tick.")
        except (KeyError, ValueError) as e:
            self.logger.error(f"Error processing tick data: {e}")

 
    async def connect_and_subscribe(self):
        retry_delay = 1
        max_retry_delay = 32
        max_retries = 10
        retries = 0

        while retries < max_retries:
            try:
                self.api.start_websocket(
                    order_update_callback=self.event_handler_order_update,
                    subscribe_callback=self.event_handler_feed_update,
                    socket_open_callback=self.open_callback,
                    socket_close_callback=self.close_callback
                )
                await self.wait_for_feed_open(timeout=30)
                self.logger.info("WebSocket connected successfully.")
                
                await self.manage_subscriptions('add', 'MCX|432294')
                #await self.manage_subscriptions('add', 'NSE|26009')
                # await self.manage_subscriptions('add', 'NSE|26000')
                
                retry_delay = 1
                retries = 0
                await self.monitor_connection()
            except Exception as e:
                self.logger.error(f"WebSocket connection error: {e}")
                retries += 1
                self.logger.info(f"Reconnecting in {retry_delay} seconds... (Attempt {retries}/{max_retries})")
                await asyncio.sleep(retry_delay)
                retry_delay = min(retry_delay * 2, max_retry_delay)

        if retries >= max_retries:
            self.logger.error("Max retries reached. Exiting.")
            raise Exception("Max retries reached")

    async def manage_subscriptions(self, command, subscription):
        token = subscription.split('|')[1]

        if command == 'add':
            if subscription not in self.active_subscriptions:
                self.api.subscribe([subscription])
                self.active_subscriptions.add(subscription)
                self.create_ring_buffers([token])
                self.create_resampled_buffers([token], self.VALID_TIMEFRAMES)
                self.logger.info(f"Subscribed to {subscription}")
                for timeframe in self.VALID_TIMEFRAMES:
                    await self.set_resampling(token, timeframe, True)
                self.logger.info(f"Resampling enabled for token {token} for all valid timeframes.")
            else:
                self.logger.warning(f"Already subscribed to {subscription}")
        elif command == 'remove':
            if subscription in self.active_subscriptions:
                self.api.unsubscribe([subscription])
                self.active_subscriptions.remove(subscription)
                self.logger.info(f"Unsubscribed from {subscription}")
            else:
                self.logger.warning(f"Not subscribed to {subscription}")

    async def wait_for_feed_open(self, timeout):
        start_time = asyncio.get_event_loop().time()
        while not self.feed_opened:
            if asyncio.get_event_loop().time() - start_time > timeout:
                raise TimeoutError("Timed out waiting for feed to open")
            await asyncio.sleep(1)

    async def monitor_connection(self):
        while True:
            if not self.feed_opened:
                self.logger.warning("Feed closed unexpectedly. Reconnecting...")
                raise Exception("Feed closed")
            await asyncio.sleep(5)

    def close_callback(self):
        self.feed_opened = False
        self.logger.warning("WebSocket connection closed.")
        self.logger.info("Attempting to reconnect...")

    def open_callback(self):
        if not self.feed_opened:
            self.feed_opened = True
            self.logger.info('Feed Opened')
        else:
            self.logger.warning('Feed Opened callback called multiple times.')

    def event_handler_order_update(self, data):
        self.logger.info(f"Order update: {data}")

    async def run(self):
        await asyncio.gather(
            self.connect_and_subscribe()            
        )

class DataProcessor:
    def __init__(self, tick_collector):
        self.tick_collector = tick_collector
        self.logger = logging.getLogger(__name__)
        self.last_processed_time = {}
        self.IDLE_THRESHOLD = timedelta(minutes=1)  # Adjust this as needed

    # async def ohlc_resampling(self):
    #     async with self.tick_collector.processing_lock:
    #         for token, ticks in self.tick_collector.ring_buffers.items():
    #             if not ticks:
    #                 self.logger.info(f"No ticks available for {token}")
    #                 continue
                
    #             # Create DataFrame from ticks
    #             df = pd.DataFrame(list(ticks))
    #             df['tt'] = pd.to_datetime(df['tt'])
    #             df.set_index('tt', inplace=True)
                
    #             #self.logger.info(f"Processing ticks for token {token}. Tick count: {len(df)}")

    #             # Iterate over each timeframe for OHLC resampling
    #             for timeframe in self.tick_collector.VALID_TIMEFRAMES:
    #                 if self.tick_collector.resampling_enabled[token].get(timeframe, False):
    #                     #self.logger.info(f"Resampling for token {token} at timeframe {timeframe}")

    #                     resampled = df['ltp'].resample(timeframe).ohlc()
                        
    #                     # Only update if we have valid resampled data
    #                     if not resampled.empty:
    #                         # Convert resampled data to dict and store in the resampled_buffers
    #                         resampled_records = resampled.reset_index().to_dict('records')
    #                         self.tick_collector.resampled_buffers[token][timeframe] = deque(
    #                             resampled_records,
    #                             maxlen=self.tick_collector.RING_BUFFER_RESAMPLE_SIZE
    #                         )
    #                         #self.logger.info(f"Updated resampled buffer for token {token} at timeframe {timeframe}. Resampled count: {len(resampled_records)}")
    #                     else:
    #                         self.logger.info(f"No resampled data for token {token} at timeframe {timeframe}")
    #                 else:
    #                     self.logger.info(f"Resampling not enabled for {token} at {timeframe}")
    async def ohlc_resampling(self):
        async with self.tick_collector.processing_lock:
            current_time = datetime.now()
            for token, ticks in self.tick_collector.ring_buffers.items():
                last_tick_time = self.tick_collector.last_tick_time.get(token)
                
                if last_tick_time is None:
                    self.logger.info(f"No last tick time available for {token}")
                    continue
                
                time_since_last_tick = current_time - last_tick_time
                
                if time_since_last_tick > self.IDLE_THRESHOLD:
                    self.logger.info(f"Token {token} idle for {time_since_last_tick}. Will process on next tick.")
                    continue
                
                if not ticks:
                    self.logger.info(f"No ticks available for {token}")
                    continue
                
                df = pd.DataFrame(list(ticks))
                df['tt'] = pd.to_datetime(df['tt'])
                df.set_index('tt', inplace=True)
                
                for timeframe in self.tick_collector.VALID_TIMEFRAMES:
                    if self.tick_collector.resampling_enabled[token].get(timeframe, False):
                        last_processed = self.last_processed_time.get((token, timeframe))
                        
                        if last_processed is None or last_tick_time > last_processed:
                            resampled = df['ltp'].resample(timeframe).ohlc().dropna()
                            
                            if not resampled.empty:
                                # Merge with existing data without filling gaps
                                existing_data = self.tick_collector.resampled_buffers[token].get(timeframe, deque())
                                existing_df = pd.DataFrame(list(existing_data)).set_index('tt') if existing_data else pd.DataFrame()
                                
                                if not existing_df.empty:
                                    # Remove any overlapping data
                                    existing_df = existing_df[existing_df.index < resampled.index[0]]
                                
                                # Concatenate existing data with new data
                                combined_df = pd.concat([existing_df, resampled])
                                
                                # Convert back to records and update the buffer
                                resampled_records = combined_df.reset_index().to_dict('records')
                                self.tick_collector.resampled_buffers[token][timeframe] = deque(
                                    resampled_records,
                                    maxlen=self.tick_collector.RING_BUFFER_RESAMPLE_SIZE
                                )
                                
                                self.last_processed_time[(token, timeframe)] = current_time
                                self.logger.info(f"Updated resampled buffer for {token} at {timeframe}. Resampled count: {len(resampled_records)}")
                            else:
                                self.logger.info(f"No resampled data for {token} at {timeframe}")
                        else:
                            self.logger.info(f"No new data for {token} at {timeframe} since last processing.")
                            
    async def process_data(self):
        #self.logger.info("DataProcessor: process_data method started")
        while True:
            #self.logger.info("DataProcessor: Starting OHLC resampling process...")
            await self.ohlc_resampling()
            await asyncio.sleep(1)  # Adjust the sleep time as needed

    def get_resampled_buffer_contents(self, token=None, timeframe=None):
    
        if token is None and timeframe is None:
            return {t: {tf: list(b) for tf, b in buffers.items()} 
                    for t, buffers in self.tick_collector.resampled_buffers.items()}
        elif token is not None and timeframe is None:
            return {tf: list(b) for tf, b in self.tick_collector.resampled_buffers.get(token, {}).items()}
        elif token is not None and timeframe is not None:
            return list(self.tick_collector.resampled_buffers.get(token, {}).get(timeframe, deque()))
        else:  # token is None and timeframe is not None
            return {t: list(buffers.get(timeframe, deque())) 
                    for t, buffers in self.tick_collector.resampled_buffers.items()}

    async def run(self):
        await asyncio.gather(
            self.process_data()            
        )
        
import nest_asyncio
# Global variable to store the DataProcessor instance
global_data_processor = None

async def main():
    global global_data_processor
    collector = TickCollector()
    processor = DataProcessor(collector)
    global_data_processor = processor
    
    await asyncio.gather(collector.run(), processor.run())

loop = asyncio.get_event_loop()
loop.set_debug(True)
if loop.is_running():
    nest_asyncio.apply()
asyncio.create_task(main())
if not loop.is_running():
    try:
        loop.run_forever()
    except KeyboardInterrupt:
        logger.info("Received exit signal. Cleaning up...")
    finally:
        loop.close()
        logger.info("Event loop closed. Exiting.")

In [ ]:
# Make sure to run this cell after the main() function has started

# Function to get resampled buffer contents
def get_buffer_contents(token=None, timeframe=None):
    global global_data_processor
    if global_data_processor is None:
        return "DataProcessor not initialized yet. Please wait a moment and try again."
    return global_data_processor.get_resampled_buffer_contents(token, timeframe)

# Examples of usage:

# Get all resampled buffer contents
all_contents = get_buffer_contents()
print("All contents:")
for token, timeframes in all_contents.items():
    print(f"Token {token}:")
    for tf, data in timeframes.items():
        print(f"  Timeframe {tf}: {len(data)} entries")
        if data:
            print(f"    complete  entry: {data}")

# Get contents for a specific token (e.g., '432294')
token_contents = get_buffer_contents(token='432294')
print("\nContents for token 432294:")
for tf, data in token_contents.items():
    print(f"  Timeframe {tf}: {len(data)} entries")
    if data:
        print(f"    Latest entry: {data[-1]}")

# Get contents for a specific token and timeframe
specific_contents = get_buffer_contents(token='432294', timeframe='1min')
print("\nContents for token 432294, 1min timeframe:")
print(f"  {len(specific_contents)} entries")
if specific_contents:
    print(f"    Latest entry: {specific_contents[-1]}")

# Get contents for a specific timeframe across all tokens
timeframe_contents = get_buffer_contents(timeframe='5min')
print("\nContents for 5min timeframe:")
for token, data in timeframe_contents.items():
    print(f"  Token {token}: {len(data)} entries")
    if data:
        print(f"    Latest entry: {data[-1]}")